In [181]:
import sqlite3
import pandas as pd

In [182]:
# define connection and cursor
conn = sqlite3.connect('store_transactions.db')
c = conn.cursor()

In [183]:
#Temporary tables -- gets deleted when the current session is ended
create_temptable = """
CREATE TEMPORARY TABLE purchases_store21 AS
    SELECT * 
    FROM purchases
    WHERE store_id = 21
"""
c.execute(create_temptable)
pd.read_sql_query("SELECT * FROM purchases_store21", conn)

,purchase_id,store_id,units_ordered,unit_price
0,4,21,55,16.40
1,5,21,63,14.70
2,54,21,6,3.67


In [184]:
#Sort and Filter
filter = """
SELECT purchase_id, unit_price, units_ordered 
FROM purchases
--WHERE store_id = 21
WHERE store_id IN (21,64) AND unit_price > 10
ORDER BY unit_price DESC;
"""
pd.read_sql_query(filter, conn)

,purchase_id,unit_price,units_ordered
0,23,21.2,2
1,4,16.4,55
2,5,14.7,63


In [185]:
#wildcard for prefix/suffix/fill. eg. %pizza, pizza%, %pizza%, S%E (slow)
filter = """
SELECT * FROM stores
WHERE brand LIKE 'br%3';
"""
pd.read_sql_query(filter, conn)


,store_id,location,brand
0,64,"Iowa City, IA",brand33


In [186]:
#Maths operations
filter = """
SELECT purchase_id, unit_price, units_ordered, 
unit_price*units_ordered AS total_cost
FROM purchases
WHERE total_cost > 300
"""
pd.read_sql_query(filter, conn)

,purchase_id,unit_price,units_ordered,total_cost
0,4,16.4,55,902.0
1,5,14.7,63,926.1
2,65,19.3,16,308.8


In [187]:
#aggregate functions
filter = """
SELECT COUNT(DISTINCT purchase_id) AS total_transactions, AVG(unit_price) AS avg_price, SUM(units_ordered) AS total_orders
FROM purchases
WHERE store_id=21
"""
pd.read_sql_query(filter, conn)

,total_transactions,avg_price,total_orders
0,3,11.59,124


In [188]:
#Grouping and filter
filter = """
SELECT store_id, COUNT(DISTINCT purchase_id) AS total_transactions
FROM purchases
WHERE store_id>50
GROUP BY store_id
HAVING total_transactions>1; --WHERE works with rows; HAVING works with groups
"""
pd.read_sql_query(filter, conn)

,store_id,total_transactions
0,95,2


In [189]:
conn.commit()
conn.close()